In [1]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

dataset_path = "asl_dataset"
img_width, img_height = 160, 160
num_classes = 36

# Create an ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

# Load and preprocess the training data
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=30,
    class_mode='categorical',
    subset='training'
)

# Load and preprocess the validation data
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=30,
    class_mode='categorical',
    subset='validation'
)

2023-05-28 14:08:01.807115: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 2012 images belonging to 36 classes.
Found 503 images belonging to 36 classes.


In [2]:

# Define the CNN model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

2023-05-28 14:08:12.108758: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

# Save the trained model
model.save("sign_language_model.h5")

Epoch 1/10
67/67 [==============================] - 48s 715ms/step - loss: 2.5781 - accuracy: 0.3522 - val_loss: 0.8208 - val_accuracy: 0.7437
Epoch 2/10
67/67 [==============================] - 39s 576ms/step - loss: 0.8730 - accuracy: 0.7543 - val_loss: 0.4307 - val_accuracy: 0.8604
Epoch 3/10
67/67 [==============================] - 33s 491ms/step - loss: 0.4291 - accuracy: 0.8668 - val_loss: 0.4186 - val_accuracy: 0.8375
Epoch 4/10
67/67 [==============================] - 41s 612ms/step - loss: 0.2570 - accuracy: 0.9173 - val_loss: 0.3979 - val_accuracy: 0.8542
Epoch 5/10
67/67 [==============================] - 33s 496ms/step - loss: 0.2598 - accuracy: 0.9273 - val_loss: 0.3682 - val_accuracy: 0.8750
Epoch 6/10
67/67 [==============================] - 35s 522ms/step - loss: 0.1694 - accuracy: 0.9445 - val_loss: 0.4710 - val_accuracy: 0.8646
Epoch 7/10
67/67 [==============================] - 44s 657ms/step - loss: 0.1342 - accuracy: 0.9531 - val_loss: 0.3855 - val_accuracy: 0.8729

In [4]:
# Load the saved model
from keras.models import load_model

model = load_model("sign_language_model.h5")

# Load and preprocess the test data
test_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Found 503 images belonging to 36 classes.
16/16 [==============================] - 3s 190ms/step - loss: 0.4458 - accuracy: 0.8946
Test Loss: 0.4457612633705139
Test Accuracy: 0.8946322202682495


In [5]:
# Predict labels for the test data
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Get the ground truth labels
true_labels = test_generator.classes

# Calculate precision, recall, and F1 score
from sklearn.metrics import classification_report

print(classification_report(true_labels, predicted_labels))

16/16 [==============================] - 3s 146ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.08      0.07      0.08        14
           2       0.05      0.07      0.06        14
           3       0.07      0.07      0.07        14
           4       0.00      0.00      0.00        14
           5       0.00      0.00      0.00        14
           6       0.00      0.00      0.00        14
           7       0.07      0.07      0.07        14
           8       0.21      0.21      0.21        14
           9       0.00      0.00      0.00        14
          10       0.07      0.07      0.07        14
          11       0.00      0.00      0.00        14
          12       0.00      0.00      0.00        14
          13       0.00      0.00      0.00        14
          14       0.00      0.00      0.00        14
          15       0.14      0.14      0.14        14
          16       0.00   